# **Bioinformatics Project - Computational Drug Discovery [Part 2] pIC50**

Nusrat Jahan

In this Jupyter notebook, we will be building a real-life data science project, we will be building a machine learning model using the CheMBL bioactivity data.
 
In **Part 2**, we will be performing Descriptor Calculation and Exploratory Data Analysis.

---

## **Install conda and rdkit**

In [1]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2022-08-24 20:14:59--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M  76.1MB/s    in 1.1s    

2022-08-24 20:15:00 (76.1 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.

## **Load bioactivity data**

In [2]:
import pandas as pd

In [3]:
! pwd

/content


In [16]:
df = pd.read_csv('/content/bioactivity_preprocessed_data_ERA.csv')
df

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL431611,Oc1ccc2c(c1)S[C@H](C1CCCC1)[C@H](c1ccc(OCCN3CC...,2.5
1,CHEMBL316132,Oc1ccc2c(c1)S[C@H](C1CCCCCC1)[C@H](c1ccc(OCCN3...,7.5
2,CHEMBL304552,Oc1ccc([C@H]2Sc3cc(O)ccc3O[C@H]2c2ccc(OCCN3CCC...,3.1
3,CHEMBL85881,Oc1ccc2c(c1)S[C@H](CC1CCCCC1)[C@H](c1ccc(OCCN3...,3.9
4,CHEMBL85536,Oc1ccc2c(c1)S[C@H](Cc1ccccc1)[C@H](c1ccc(OCCN3...,7.4
...,...,...,...
3904,CHEMBL4857695,Cc1cccc(CN(CC2(c3ccccc3F)CCCC2)c2ccc(OCCn3cc(C...,2205.0
3905,CHEMBL4869947,Cc1cccc(CN(CC2(c3ccccc3F)CCCC2)c2ccc(OCCn3cc(C...,329.0
3906,CHEMBL4850252,Cc1cccc(CN(CC2(c3ccccc3F)CCCC2)c2ccc(OCCn3cc(C...,261.0
3907,CHEMBL4858474,Cc1cccc(CN(CC2(c3ccccc3F)CCCC2)c2ccc(OCCn3cc(C...,199.0


## **Calculate Lipinski descriptors**
Christopher Lipinski, a scientist at Pfizer, came up with a set of rule-of-thumb for evaluating the **druglikeness** of compounds. Such druglikeness is based on the Absorption, Distribution, Metabolism and Excretion (ADME) that is also known as the pharmacokinetic profile. Lipinski analyzed all orally active FDA-approved drugs in the formulation of what is to be known as the **Rule-of-Five** or **Lipinski's Rule**.

The Lipinski's Rule stated the following:
* Molecular weight < 500 Dalton
* Octanol-water partition coefficient (LogP) < 5 (solubility in water/Hydrophobicity/Hydrophilicity)
* Hydrogen bond donors < 5
* Hydrogen bond acceptors < 10 

### **Import libraries**

In [17]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [20]:
df

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL431611,Oc1ccc2c(c1)S[C@H](C1CCCC1)[C@H](c1ccc(OCCN3CC...,2.5
1,CHEMBL316132,Oc1ccc2c(c1)S[C@H](C1CCCCCC1)[C@H](c1ccc(OCCN3...,7.5
2,CHEMBL304552,Oc1ccc([C@H]2Sc3cc(O)ccc3O[C@H]2c2ccc(OCCN3CCC...,3.1
3,CHEMBL85881,Oc1ccc2c(c1)S[C@H](CC1CCCCC1)[C@H](c1ccc(OCCN3...,3.9
4,CHEMBL85536,Oc1ccc2c(c1)S[C@H](Cc1ccccc1)[C@H](c1ccc(OCCN3...,7.4
...,...,...,...
3904,CHEMBL4857695,Cc1cccc(CN(CC2(c3ccccc3F)CCCC2)c2ccc(OCCn3cc(C...,2205.0
3905,CHEMBL4869947,Cc1cccc(CN(CC2(c3ccccc3F)CCCC2)c2ccc(OCCn3cc(C...,329.0
3906,CHEMBL4850252,Cc1cccc(CN(CC2(c3ccccc3F)CCCC2)c2ccc(OCCn3cc(C...,261.0
3907,CHEMBL4858474,Cc1cccc(CN(CC2(c3ccccc3F)CCCC2)c2ccc(OCCn3cc(C...,199.0


Now, let's combine the 2 DataFrame

### **Convert IC50 to pIC50**
To allow **IC50** data to be more uniformly distributed, we will convert **IC50** to the negative logarithmic scale which is essentially **-log10(IC50)**.

This custom function pIC50() will accept a DataFrame as input and will:
* Take the IC50 values from the ``standard_value`` column and converts it from nM to M by multiplying the value by 10$^{-9}$
* Take the molar value and apply -log10
* Delete the ``standard_value`` column and create a new ``pIC50`` column

In [21]:
# https://github.com/chaninlab/estrogen-receptor-alpha-qsar/blob/master/02_ER_alpha_RO5.ipynb

import numpy as np

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)
        
    return x

Point to note: Values greater than 100,000,000 will be fixed at 100,000,000 otherwise the negative logarithmic value will become negative.

In [22]:
df.standard_value.describe()

count    3.909000e+03
mean     2.202800e+04
std      1.535920e+05
min      2.000000e-03
25%      7.500000e+00
50%      1.720000e+02
75%      5.000000e+03
max      5.000000e+06
Name: standard_value, dtype: float64

In [23]:
-np.log10( (10**-9)* 100000000 )

1.0

In [24]:
-np.log10( (10**-9)* 10000000000 )

-1.0

In [25]:
# to keep the value positive
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', 1)
        
    return x

We will first apply the norm_value() function so that the values in the standard_value column is normalized.

In [26]:
df_norm = norm_value(df)
df_norm

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # This is added back by InteractiveShellApp.init_path()


,molecule_chembl_id,canonical_smiles,standard_value_norm
0,CHEMBL431611,Oc1ccc2c(c1)S[C@H](C1CCCC1)[C@H](c1ccc(OCCN3CC...,2.5
1,CHEMBL316132,Oc1ccc2c(c1)S[C@H](C1CCCCCC1)[C@H](c1ccc(OCCN3...,7.5
2,CHEMBL304552,Oc1ccc([C@H]2Sc3cc(O)ccc3O[C@H]2c2ccc(OCCN3CCC...,3.1
3,CHEMBL85881,Oc1ccc2c(c1)S[C@H](CC1CCCCC1)[C@H](c1ccc(OCCN3...,3.9
4,CHEMBL85536,Oc1ccc2c(c1)S[C@H](Cc1ccccc1)[C@H](c1ccc(OCCN3...,7.4
...,...,...,...
3904,CHEMBL4857695,Cc1cccc(CN(CC2(c3ccccc3F)CCCC2)c2ccc(OCCn3cc(C...,2205.0
3905,CHEMBL4869947,Cc1cccc(CN(CC2(c3ccccc3F)CCCC2)c2ccc(OCCn3cc(C...,329.0
3906,CHEMBL4850252,Cc1cccc(CN(CC2(c3ccccc3F)CCCC2)c2ccc(OCCn3cc(C...,261.0
3907,CHEMBL4858474,Cc1cccc(CN(CC2(c3ccccc3F)CCCC2)c2ccc(OCCn3cc(C...,199.0


In [27]:
df_norm.standard_value_norm.describe()

count    3.909000e+03
mean     2.202800e+04
std      1.535920e+05
min      2.000000e-03
25%      7.500000e+00
50%      1.720000e+02
75%      5.000000e+03
max      5.000000e+06
Name: standard_value_norm, dtype: float64

In [30]:
df_final = pIC50(df_norm)
df_final

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  del sys.path[0]


,molecule_chembl_id,canonical_smiles,pIC50
0,CHEMBL431611,Oc1ccc2c(c1)S[C@H](C1CCCC1)[C@H](c1ccc(OCCN3CC...,8.602060
1,CHEMBL316132,Oc1ccc2c(c1)S[C@H](C1CCCCCC1)[C@H](c1ccc(OCCN3...,8.124939
2,CHEMBL304552,Oc1ccc([C@H]2Sc3cc(O)ccc3O[C@H]2c2ccc(OCCN3CCC...,8.508638
3,CHEMBL85881,Oc1ccc2c(c1)S[C@H](CC1CCCCC1)[C@H](c1ccc(OCCN3...,8.408935
4,CHEMBL85536,Oc1ccc2c(c1)S[C@H](Cc1ccccc1)[C@H](c1ccc(OCCN3...,8.130768
...,...,...,...
3904,CHEMBL4857695,Cc1cccc(CN(CC2(c3ccccc3F)CCCC2)c2ccc(OCCn3cc(C...,5.656591
3905,CHEMBL4869947,Cc1cccc(CN(CC2(c3ccccc3F)CCCC2)c2ccc(OCCn3cc(C...,6.482804
3906,CHEMBL4850252,Cc1cccc(CN(CC2(c3ccccc3F)CCCC2)c2ccc(OCCn3cc(C...,6.583359
3907,CHEMBL4858474,Cc1cccc(CN(CC2(c3ccccc3F)CCCC2)c2ccc(OCCn3cc(C...,6.701147


In [33]:
df_final.to_csv('bioactivity_pIC50_ERA.csv', index=False)

In [34]:
df_final.pIC50.describe()

count    3909.000000
mean        6.752992
std         1.667116
min         2.301030
25%         5.301030
50%         6.764472
75%         8.124939
max        11.698970
Name: pIC50, dtype: float64